In [1]:
import pandas as pd
import numpy as np
from collections import Counter

import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

In [43]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 30)

## Data Set Information
This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

---

## Attribute Information
- **InvoiceNo**: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'C', it indicates a cancellation.
- **StockCode**: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- **Description**: Product (item) name. Nominal.
- **Quantity**: The quantity of each product (item) per transaction. Numeric.
- **InvoiceDate**: Invoice date and time. Numeric, representing the day and time when each transaction was generated.
- **UnitPrice**: Unit price. Numeric, representing the product price per unit in sterling.
- **CustomerID**: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- **Country**: Country name. Nominal, representing the name of the country where each customer resides.


In [2]:
raw_data = pd.read_csv('../data/retail.csv', encoding='ISO-8859-1')
display(raw_data.head(10))

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/10 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/10 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/10 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/10 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/10 8:34,1.69,13047.0,United Kingdom


In [3]:
raw_product_descriptions = pd.read_csv('../data/main_product_descriptions.csv')
display(raw_product_descriptions.head(10))

,item_family,item_family_description,category
0,10002,INFLATABLE POLITICAL GLOBE,GIFT ITEMS
1,10080,GROOVY CACTUS INFLATABLE,TOYS
2,10120,DOGGY RUBBER,TOYS
3,10123,HEARTS WRAPPING TAPE,STATIONERY
4,10124,BOOKCOVER TAPE,STATIONERY
5,10125,MINI FUNKY DESIGN TAPES,CRAFTS
6,10133,COLOURING PENCILS BROWN TUBE,STATIONERY
7,10135,COLOURING PENCILS BROWN TUBE,STATIONERY
8,11001,ASSTD DESIGN RACING CAR PEN,STATIONERY
9,15030,FAN BLACK FRAME,LIGHTING


### Item Family

In [91]:
item_family = raw_product_descriptions.copy()
item_family = item_family.rename(columns={
    'item_family': 'item_family_id',
    'category': 'item_category'
})

item_family['item_family_id'] = item_family['item_family_id'].astype('Int64')
item_family['item_family_description'] = item_family['item_family_description'].astype(str)
item_family['item_category'] = item_family['item_category'].astype(str)
display(item_family.head(10))

,item_family_id,item_family_description,item_category
0,10002,INFLATABLE POLITICAL GLOBE,GIFT ITEMS
1,10080,GROOVY CACTUS INFLATABLE,TOYS
2,10120,DOGGY RUBBER,TOYS
3,10123,HEARTS WRAPPING TAPE,STATIONERY
4,10124,BOOKCOVER TAPE,STATIONERY
5,10125,MINI FUNKY DESIGN TAPES,CRAFTS
6,10133,COLOURING PENCILS BROWN TUBE,STATIONERY
7,10135,COLOURING PENCILS BROWN TUBE,STATIONERY
8,11001,ASSTD DESIGN RACING CAR PEN,STATIONERY
9,15030,FAN BLACK FRAME,LIGHTING


### Transactions

In [166]:
import hashlib

def generate_hash(row, columns):
    combined_string = "_".join(str(row[col]) for col in columns)
    return hashlib.md5(combined_string.encode()).hexdigest()

In [167]:
transactions = raw_data.copy()

transactions = transactions.rename(columns={
    'InvoiceNo': 'invoice_id',
    'StockCode': 'item_id',
    'Description': 'item_description',
    'Quantity': 'quantity_amount',
    'InvoiceDate': 'event_timestamp_invoiced_at',
    'UnitPrice': 'unit_price_eur',
    'CustomerID': 'customer_id',
    'Country': 'country_name'
})

transactions['invoice_id'] = transactions['invoice_id'].astype(str)
transactions['item_id'] = transactions['item_id'].astype(str)
transactions['item_description'] = transactions['item_description'].astype(str)
transactions['quantity_amount'] = transactions['quantity_amount'].astype(int)
transactions['event_timestamp_invoiced_at'] = pd.to_datetime(transactions['event_timestamp_invoiced_at'])
transactions['unit_price_eur'] = transactions['unit_price_eur'].astype(float)
transactions['customer_id'] = transactions['customer_id'].astype('Int64')
transactions['country_name'] = transactions['country_name'].astype(str)
transactions.index.name = 'transaction_id'
transactions.reset_index(inplace=True)
transactions['transaction_id'] = transactions['transaction_id'].astype('Int64')
transactions['item_uuid'] = transactions.apply(lambda row: generate_hash(row, ['item_id', 'item_description']), axis=1)
transactions['is_return'] = transactions['quantity_amount'] < 0

transactions.head(10)

/var/folders/hv/ckh3m6gn1sd45q2qctrqcwzh0000gn/T/ipykernel_68502/3931827383.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  transactions['event_timestamp_invoiced_at'] = pd.to_datetime(transactions['event_timestamp_invoiced_at'])


,transaction_id,invoice_id,item_id,item_description,quantity_amount,event_timestamp_invoiced_at,unit_price_eur,customer_id,country_name,item_uuid,is_return
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,a5eeac374da2111031f7519f81943391,False
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,fe7a790952c55d610228175df6bc6a3c,False
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,28c1dc0613efda1aeb0113274a9a8dde,False
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,70c1999f8fb610f63be227d65201f501,False
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,cec12c5a55ddfa8fb7eb3ae3b24c8c46,False
5,5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850,United Kingdom,6c0f11fb00cb0402b6069bc5ce24f417,False
6,6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850,United Kingdom,4aeede5c1953565d784ca75e7a469a70,False
7,7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,6ea27d45ef36140165e860da38820cca,False
8,8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,63b9a0eba79590ef9119a382140dc6d9,False
9,9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,412535c0333f0819242d23289f170e5d,False


### Items

In [94]:
items = transactions.groupby(['item_uuid','item_id', 'item_description']).count()[[]].reset_index()
items.sample(20)

,item_uuid,item_id,item_description
3903,adae585a2c04b39fdb8578f5e5aff9d1,gift_0001_30,nan
3168,8ce18ec58c16921369d48fd79c1a8c62,85175,wrongly sold sets
5353,eeb531c6def558bd9e21d4c36bfcfea8,23625,SET 6 CARDS SPARKLY REINDEER 17262
3483,9b2ab2a90f7fe9556209656d257cf3e7,23296,SET OF 6 TEA TIME BAKING CASES
3858,abb27362910e598bb0db01cadacd8568,23116,Damaged
4465,c83fc68a57ef3f99f878cea53f153e4c,22303,COFFEE MUG APPLES DESIGN
2804,7d0ce428665050a687dac1c10ba00bd8,20772,GARDEN PATH JOURNAL
2332,69101f65d24a9a948ed9f4b1888d10fc,84927E,FLOWERS TILE HOOK
5502,f53168767d1de8657ebb9c4a7dda61cd,20681,MIA
3721,a54d77b375518b80bdee8ef925f202b3,20749,ASSORTED COLOUR MINI CASES


In [95]:
items['is_operational_item'] = (items['item_id'].str.len() < 5) | (items['item_id'].str.contains('gift', case=False))
items['item_family_id'] = items['item_id'].str.extract(r'^(\d+)', expand=False)
items['item_variant'] = items['item_id'].str.extract(r'(\D+)$', expand=False).fillna('')
items['item_variant'] = np.where(items['item_id'] == items['item_family_id'],  np.nan, items['item_variant'])

items.head()

,item_uuid,item_id,item_description,is_operational_item,item_family_id,item_variant
0,00101644c59f5ca2a40eba1ec330267f,85114B,IVORY ENCHANTED FOREST PLACEMAT,False,85114,B
1,0011274a066e1cd254cbb1f466b8fc70,23457,LARGE PARLOUR FRAME,False,23457,NaN
2,00337bc60cace50c98618a90a70682ca,21042,RED RETROSPOT APRON,False,21042,NaN
3,0041175d5926046b2323f63772a8c706,84789,nan,False,84789,NaN
4,006e6672caa869d2f012cca839a92dc0,21054,nan,False,21054,NaN


#### Set items additional info flags

In [96]:
selected_items = items[items['is_operational_item'] == False]

selected_items['is_all_uppercase'] = selected_items['item_description'].str.isupper()
selected_items['description_len'] = selected_items['item_description'].str.len()
selected_items = selected_items.sort_values(by=['item_id', 'description_len'], ascending=[True, False])
selected_items['description_order'] = selected_items.groupby('item_id').cumcount() + 1

selected_items.head()

/var/folders/hv/ckh3m6gn1sd45q2qctrqcwzh0000gn/T/ipykernel_68502/3455374893.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_items['is_all_uppercase'] = selected_items['item_description'].str.isupper()
/var/folders/hv/ckh3m6gn1sd45q2qctrqcwzh0000gn/T/ipykernel_68502/3455374893.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_items['description_len'] = selected_items['item_description'].str.len()


,item_uuid,item_id,item_description,is_operational_item,item_family_id,item_variant,is_all_uppercase,description_len,description_order
4947,dcf485d95dacdff7f8bd0b2c77bd427e,10002,INFLATABLE POLITICAL GLOBE,False,10002,NaN,True,27,1
4723,d369bed0691da2bf94089200fc5908e9,10002,nan,False,10002,NaN,False,3,2
1031,2d871dda5cd5ec3aced134048e3c5d86,10080,GROOVY CACTUS INFLATABLE,False,10080,NaN,True,24,1
594,1a2362ac03e68264094428758062ee5b,10080,check,False,10080,NaN,False,5,2
1692,4b436a57abd3886b69b52e4121480b1f,10080,nan,False,10080,NaN,False,3,3


In [97]:
weird_items = selected_items[((selected_items['description_order'] > 1) & (selected_items['description_len'] < 13)) | ~selected_items['is_all_uppercase']]
description_list = weird_items['item_description'].drop_duplicates().tolist()
print(description_list)

is_unknown_item = []
is_error_item = []
is_special_item = []
is_modification_item = []

for description in description_list:
    description_lower = description.lower()
    if any(word in description_lower for word in [
        'check', 'damage', 'wet', 'lost', 'found', 'error', 'wrong', 'faulty', 'mouldy', 'smashed', 'broken', 'crush', 'crack', 'unsaleable', 'missing', 'throw away'
    ]):
        is_error_item.append(description)
    elif any(word in description_lower for word in [
        'dotcom', 'amazon', 'fba', 'ebay', 'john lewis', 'showroom', 'voucher', 'cordial jug', 'website'
    ]):
        is_special_item.append(description)
    elif any(word in description_lower for word in [
        'adjust', 'sample', 'sold as', 'mailout', 'allocate', 'temp', 'credit', 'sale', 're-adjustment', 'label mix up'
    ]):
        is_modification_item.append(description)


is_unknown_item = ['?','??','???','Incorrect stock entry.',"can't find",'nan',None]

is_error_item += [
    'on cargo order', 'test', 'barcode problem', 'mixed up', 'michel oops', 'printing smudges/thrown away', 
    'had been put aside', 'rusty thrown away', 'incorrectly made-thrown away.', 'Breakages', 'counted', 
    'Had been put aside.', 'returned', 'thrown away', 'mystery! Only ever imported 1800', 'Dagamed', 
    'code mix up? 84930', 'Printing smudges/thrown away', 'came coded as 20713', 'incorrect stock entry.',
    "thrown away-can't sell",'Thrown away-rusty','Thrown away.','Given away','historic computer difference?....se',
    'alan hodge cant mamage this section',"thrown away-can't sell.", 'label mix up','sold in set?','mix up with c'
]

is_special_item += [
    'MIA', '?display?', 'Amazon Adjustment', 'Lighthouse Trading zero invc incorr', 
    'Dotcomgiftshop Gift Voucher £100.00', 'sold as set?', 
    'High Resolution Image', 'John Lewis','Bank Charges','Next Day Carriage'
]

is_modification_item += [
    'Adjustment', 'OOPS ! adjustment', 'reverse 21/5/10 adjustment', 'reverse previous adjustment', 
    'marked as 23343', 'incorrectly put back into stock', 'Not rcvd in 10/11/2010 delivery', 'Display', 
    'Had been put aside.',  'sold as set by dotcom', 'add stock to allocate online orders', 
    'allocate stock for dotcom orders ta', 'for online retail orders', 'Marked as 23343'
]

unclassified_items = set(description_list) - set(is_error_item) - set(is_special_item) - set(is_modification_item)

print("is_error_item:", is_error_item)
print("is_special_item:", is_special_item)
print("is_modification_item:", is_modification_item)
print("Unclassified items:", unclassified_items)


['nan', 'check', 'damaged', 'wet/rusty', 'FLOWERS HANDBAG blue and orange', 'alan hodge cant mamage this section', 'THE KING GIFT BAG 25x24x12cm', 'found', 'Adjustment', 'ESSENTIAL BALM 3.5g TIN IN ENVELOPE', 'adjustment', 'dotcom', 'MIA', '?', 'lost in space', 'wrongly marked. 23343 in box', 'wrongly marked 23343', 'wrongly coded 23343', 'wrongly coded-23343', 'Marked as 23343', 'Found', 'damages', '??', 'damages/display', '*USB Office Mirror Ball', 'POLYESTER FILLER PAD 60x40cm', 'Amazon Adjustment', 'taig adjust no stock', 'WET/MOULDY', 'had been put aside', 'damages?', 'Damaged', 'wet', '*Boombox Ipod Classic', 'sold as set on dotcom', 'wet rusty', 'on cargo order', 'wrongly marked', '???', 'broken', 'Show Samples', 'Sold as 1 on dotcom', "Dr. Jam's Arouzer Stress Ball", "Dad's Cab Electronic Meter", 'CHECK', 'mystery! Only ever imported 1800', 'rcvd be air temp fix for dotcom sit', 're dotcom quick fix.', 'samples', '?display?', 'mixed up', 'wrong barcode', 'sold as set on dotcom 

In [98]:
items['is_unknown_item'] = items['item_description'].str.lower().isin(is_unknown_item)
items['is_error_item'] = items['item_description'].str.lower().isin(is_error_item)
items['is_special_item'] = items['item_description'].str.lower().isin(is_special_item)
items['is_modification_item'] = items['item_description'].str.lower().isin(is_modification_item)

items = items[['item_uuid', 'item_id', 'item_family_id', 'item_description', 'item_variant', 'is_operational_item', 'is_unknown_item', 'is_special_item', 'is_modification_item', 'is_error_item']]

items.loc[items['is_operational_item'], ['item_variant', 'is_unknown_item', 'is_special_item', 'is_modification_item', 'is_error_item']] = [None, False, False, False, False]
items.loc[items['is_unknown_item']] = items.loc[items['is_unknown_item']].replace('', None)
items.loc[items['is_special_item']] = items.loc[items['is_special_item']].replace('', None)
items.loc[items['is_modification_item']] = items.loc[items['is_modification_item']].replace('', None)
items.loc[items['is_error_item']] = items.loc[items['is_error_item']].replace('', None)

items.head(10)

,item_uuid,item_id,item_family_id,item_description,item_variant,is_operational_item,is_unknown_item,is_special_item,is_modification_item,is_error_item
0,00101644c59f5ca2a40eba1ec330267f,85114B,85114,IVORY ENCHANTED FOREST PLACEMAT,B,False,False,False,False,False
1,0011274a066e1cd254cbb1f466b8fc70,23457,23457,LARGE PARLOUR FRAME,NaN,False,False,False,False,False
2,00337bc60cace50c98618a90a70682ca,21042,21042,RED RETROSPOT APRON,NaN,False,False,False,False,False
3,0041175d5926046b2323f63772a8c706,84789,84789,nan,NaN,False,True,False,False,False
4,006e6672caa869d2f012cca839a92dc0,21054,21054,nan,NaN,False,True,False,False,False
5,00818d3df6cadf9dba5e0a23ec4dbb7e,21671,21671,RED SPOT CERAMIC DRAWER KNOB,NaN,False,False,False,False,False
6,00a6ebf87edbf17fbcbc558c07a444cb,21250,21250,SET OF SKULL WALL STICKERS,NaN,False,False,False,False,False
7,00b32ae49174c16cbca6cafc7bdf0834,47566B,47566,reverse previous adjustment,B,False,False,False,True,False
8,00bb997f07a1d4086532a72e5cab8ebe,23295,23295,SET OF 12 MINI LOAF BAKING CASES,NaN,False,False,False,False,False
9,00dde1d19992d625296429f8d7be620f,21715,21715,GIRLS VINTAGE TIN SEASIDE BUCKET,NaN,False,False,False,False,False


In [99]:
filtered_items = items[~items['is_operational_item'] & ~items['is_error_item'] & ~items['is_special_item'] & ~items['is_modification_item']& ~items['is_unknown_item']]
grouped = filtered_items.groupby('item_family_id')['item_description'].apply(lambda x: ' '.join(x))

def get_most_common_words(text):
    words = text.split()  
    word_counts = Counter(words)
    max_count = max(word_counts.values(), default=0)
    
    most_common_words = [word for word, count in word_counts.items() if count == max_count]
    return ' '.join(most_common_words) if most_common_words else None

main_product_descriptions = grouped.apply(get_most_common_words).reset_index()
main_product_descriptions.columns = ['item_family', 'item_family_description']


main_product_descriptions.head(10)

,item_family,item_family_description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123,HEARTS WRAPPING TAPE
4,10124,BOOKCOVER TAPE
5,10125,MINI FUNKY DESIGN TAPES
6,10133,COLOURING PENCILS BROWN TUBE
7,10135,COLOURING PENCILS BROWN TUBE
8,11001,ASSTD DESIGN RACING CAR PEN
9,15030,FAN BLACK FRAME


### Set categories with Open AI LLM

In [100]:
categories = [
    "HOME DECOR",
    "KITCHENWARE",
    "TABLEWARE",
    "GIFT ITEMS",
    "FURNITURE",
    "TOYS",
    "CANDLES",
    "JEWELRY",
    "TEXTILES",
    "GARDENING",
    "BATHROOM",
    "STATIONERY",
    "BAGS",
    "LIGHTING",
    "CRAFTS",
    "HOLIDAY",
    "OUTDOOR",
    "TOOL & DIY",
    "SPORTS",
    "STORAGE",
    "TRAVEL",
    "BEAUTY",
    "BOOKS",
    "GAMES",
    "ORGANIZERS",
    "TABLE LINEN",
    "CHILDREN",
    "CERAMICS",
    "CLOTHING",
    "PLANTS & FLOWERS",
    "OTHERS"
]

#### LLM Extraction

In [101]:
from openai import OpenAI

import openai_setup

# Create GPT Client
def initChatGPTClient():
    organization = openai_setup.conf['organization']
    project = openai_setup.conf['project']
    key = openai_setup.conf['key']

    client = OpenAI(
        api_key=key,
        organization=organization,
        project=project
    )
    return client

def llm_extract_category(client, item):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert classifier system that gives a category to each object from a commerce. Ansewer only with the category selected in uppercase. The possible categories:" + str(categories)},
            {
                "role": "user",
                "content": str(item)
            }
        ]
    )
    answer = completion.choices[0].message.content
    return answer

In [102]:
if False:
    client = initChatGPTClient()
    main_product_descriptions['category'] = main_product_descriptions['item_family_description'].apply(lambda item: llm_extract_category(client, item))
    display(main_product_descriptions)

    main_product_descriptions.to_csv('../data/main_product_descriptions.csv', index=False)

## Set modelling

#### dim_items

In [118]:
dim_items = items.copy()

dim_items['item_family_id'] = dim_items['item_family_id'].astype('Int64')

dim_items.head()

,item_uuid,item_id,item_family_id,item_description,item_variant,is_operational_item,is_unknown_item,is_special_item,is_modification_item,is_error_item
0,00101644c59f5ca2a40eba1ec330267f,85114B,85114,IVORY ENCHANTED FOREST PLACEMAT,B,False,False,False,False,False
1,0011274a066e1cd254cbb1f466b8fc70,23457,23457,LARGE PARLOUR FRAME,NaN,False,False,False,False,False
2,00337bc60cace50c98618a90a70682ca,21042,21042,RED RETROSPOT APRON,NaN,False,False,False,False,False
3,0041175d5926046b2323f63772a8c706,84789,84789,nan,NaN,False,True,False,False,False
4,006e6672caa869d2f012cca839a92dc0,21054,21054,nan,NaN,False,True,False,False,False


#### dim_items_family

In [104]:
dim_item_family = item_family.copy()

dim_item_family.head()

,item_family_id,item_family_description,item_category
0,10002,INFLATABLE POLITICAL GLOBE,GIFT ITEMS
1,10080,GROOVY CACTUS INFLATABLE,TOYS
2,10120,DOGGY RUBBER,TOYS
3,10123,HEARTS WRAPPING TAPE,STATIONERY
4,10124,BOOKCOVER TAPE,STATIONERY


#### dim_date

In [105]:
date_range = pd.date_range(start='2009-01-01', end='2025-12-31', freq='D')

dim_date = pd.DataFrame({
    'date_id': range(1, len(date_range) + 1),
    'date': date_range.date,
    'year': date_range.year,
    'month': date_range.month,
    'day': date_range.day,
    'day_of_week': date_range.strftime('%A'),
    'is_weekend': date_range.weekday >= 5 
})
display(dim_date)

,date_id,date,year,month,day,day_of_week,is_weekend
0,1,2009-01-01,2009,1,1,Thursday,False
1,2,2009-01-02,2009,1,2,Friday,False
2,3,2009-01-03,2009,1,3,Saturday,True
3,4,2009-01-04,2009,1,4,Sunday,True
4,5,2009-01-05,2009,1,5,Monday,False
...,...,...,...,...,...,...,...
6204,6205,2025-12-27,2025,12,27,Saturday,True
6205,6206,2025-12-28,2025,12,28,Sunday,True
6206,6207,2025-12-29,2025,12,29,Monday,False
6207,6208,2025-12-30,2025,12,30,Tuesday,False


#### dim_customers

In [106]:
from faker import Faker
import random
import pandas as pd

customers = transactions[['customer_id', 'country_name']].drop_duplicates().dropna().reset_index(drop=True)

faker = Faker()
countries_locale = {
    "Australia": 'en_AU', "Austria": 'de_AT', "Bahrain": 'en_US', "Belgium": 'fr_BE',
    "Brazil": 'pt_BR', "Canada": 'en_CA', "Channel Islands": 'en_GB', "Cyprus": 'el_CY',
    "Czech Republic": 'cs_CZ', "Denmark": 'da_DK', "EIRE": 'en_IE', "European Community": 'en_US',
    "Finland": 'fi_FI', "France": 'fr_FR', "Germany": 'de_DE', "Greece": 'el_GR',
    "Iceland": 'en_US', "Israel": 'he_IL', "Italy": 'it_IT', "Japan": 'ja_JP',
    "Lebanon": 'ar_SA', "Lithuania": 'lt_LT', "Malta": 'mt_MT', "Netherlands": 'nl_NL',
    "Norway": 'no_NO', "Poland": 'pl_PL', "Portugal": 'pt_PT', "RSA": 'en_US',
    "Saudi Arabia": 'ar_SA', "Singapore": 'en_US', "Spain": 'es_ES', "Sweden": 'sv_SE',
    "Switzerland": 'de_CH', "USA": 'en_US', "United Arab Emirates": 'ar_AE',
    "United Kingdom": 'en_GB', "Unspecified": 'en_US'
}

names_by_country = {}
for country, locale in countries_locale.items():
    faker = Faker(locale)
    first_names = [faker.first_name() for _ in range(1000)]
    last_names = [faker.last_name() for _ in range(1000)]
    full_names = [f"{random.choice(first_names)} {random.choice(last_names)}" for _ in range(1000)]
    names_by_country[country] = full_names

def assign_name(row):
    country = row['country_name']
    if country in names_by_country:
        return random.choice(names_by_country[country])
    return "Unknown"

customers['customer_name'] = customers.apply(assign_name, axis=1)
customers = customers.drop(columns=['country_name'])
dim_customers = customers.drop_duplicates(subset=['customer_id'], keep='first').reset_index(drop=True)
display(dim_customers.head())

,customer_id,customer_name
0,17850,Kirsty Schofield
1,13047,Bruce Woods
2,12583,Alexandre Becker
3,13748,Dennis Page
4,15100,Joyce Martin


#### dim_location

In [107]:
country_data = {
    'country_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
                   21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 
                   38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 
                   55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 
                   72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 
                   89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 
                   105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 
                   118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 
                   131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 
                   144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 
                   157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 
                   170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 
                   183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 
                   196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 
                   209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 
                   222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 
                   235, 236, 237, 238, 239, 240, 241, 242, 243, 244],
    'iso_country_code': ['AF', 'AL', 'DZ', 'AS', 'AD', 'AO', 'AI', 'AQ', 'AG', 'AR', 'AM', 
                         'AW', 'AU', 'AT', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BE', 'BZ', 
                         'BJ', 'BM', 'BT', 'BO', 'BA', 'BW', 'BV', 'BR', 'IO', 'BN', 'BG', 
                         'BF', 'BI', 'KH', 'CM', 'CA', 'CV', 'KY', 'CF', 'TD', 'CL', 'CN', 
                         'CX', 'CC', 'CO', 'KM', 'CG', 'CD', 'CK', 'CR', 'CI', 'HR', 'CU', 
                         'CY', 'CZ', 'DK', 'DJ', 'DM', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 
                         'EE', 'ET', 'FK', 'FO', 'FJ', 'FI', 'FR', 'GF', 'PF', 'TF', 'GA', 
                         'GM', 'GE', 'DE', 'GH', 'GI', 'GR', 'GL', 'GD', 'GP', 'GU', 'GT', 
                         'GN', 'GW', 'GY', 'HT', 'HM', 'VA', 'HN', 'HK', 'HU', 'IS', 'IN', 
                         'ID', 'IR', 'IQ', 'IE', 'IL', 'IT', 'JM', 'JP', 'JO', 'KZ', 'KE', 
                         'KI', 'KP', 'KR', 'KW', 'KG', 'LA', 'LV', 'LB', 'LS', 'LR', 'LY', 
                         'LI', 'LT', 'LU', 'MO', 'MK', 'MG', 'MW', 'MY', 'MV', 'ML', 'MT', 
                         'MH', 'MQ', 'MR', 'MU', 'YT', 'MX', 'FM', 'MD', 'MC', 'MN', 'MS', 
                         'MA', 'MZ', 'MM', 'NA', 'NR', 'NP', 'NL', 'AN', 'NC', 'NZ', 'NI', 
                         'NE', 'NG', 'NU', 'NF', 'MP', 'NO', 'OM', 'PK', 'PW', 'PS', 'PA', 
                         'PG', 'PY', 'PE', 'PH', 'PN', 'PL', 'PT', 'PR', 'QA', 'RE', 'RO', 
                         'RU', 'RW', 'SH', 'KN', 'LC', 'PM', 'VC', 'WS', 'SM', 'ST', 'SA', 
                         'SN', 'CS', 'SC', 'SL', 'SG', 'SK', 'SI', 'SB', 'SO', 'ZA', 'GS', 
                         'ES', 'LK', 'SD', 'SR', 'SJ', 'SZ', 'SE', 'CH', 'SY', 'TW', 'TJ', 
                         'TZ', 'TH', 'TL', 'TG', 'TK', 'TO', 'TT', 'TN', 'TR', 'TM', 'TC', 
                         'TV', 'UG', 'UA', 'AE', 'GB', 'US', 'UM', 'UY', 'UZ', 'VU', 'VE', 
                         'VN', 'VG', 'VI', 'WF', 'EH', 'YE', 'ZM', 'ZW', 'IE', 'XX', '00','EU', 'ZA'],
    'country_name': ['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 
                     'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 
                     'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 
                     'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 
                     'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 
                     'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 
                     'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 
                     'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 
                     'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 
                     'Colombia', 'Comoros', 'Congo', 'Congo, the Democratic Republic of the', 
                     'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 
                     'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 
                     'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 
                     'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 
                     'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 
                     'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 
                     'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guinea', 
                     'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 
                     'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 
                     'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 
                     'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', 
                     'Korea, Democratic People’s Republic of', 'Korea, Republic of', 'Kuwait', 
                     'Kyrgyzstan', 'Lao People’s Democratic Republic', 'Latvia', 'Lebanon', 'Lesotho', 
                     'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 
                     'North Macedonia', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 
                     'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 
                     'Mexico', 'Micronesia (Federated States of)', 'Moldova', 'Monaco', 'Mongolia', 
                     'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 
                     'Netherlands', 'Netherlands Antilles', 'New Caledonia', 'New Zealand', 'Nicaragua', 
                     'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 
                     'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestine, State of', 'Panama', 
                     'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 
                     'Portugal', 'Puerto Rico', 'Qatar', 'Reunion', 'Romania', 'Russian Federation', 
                     'Rwanda', 'Saint Helena', 'Saint Kitts and Nevis', 'Saint Lucia', 
                     'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 
                     'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 
                     'Serbia and Montenegro', 'Seychelles', 'Sierra Leone', 'Singapore', 'Slovakia', 
                     'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 
                     'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 
                     'Suriname', 'Svalbard and Jan Mayen', 'Eswatini', 'Sweden', 'Switzerland', 
                     'Syrian Arab Republic', 'Taiwan', 'Tajikistan', 'Tanzania', 'Thailand', 
                     'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 
                     'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 
                     'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'USA', 
                     'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 
                     'Venezuela', 'Vietnam', 'British Virgin Islands', 'U.S. Virgin Islands', 
                     'Wallis and Futuna', 'Western Sahara', 'Yemen', 'Zambia', 'Zimbabwe',
                     'EIRE', 'Channel Islands','Unspecified','European Community','RSA'],
    'country_code_name': ['AFGHANISTAN', 'ALBANIA', 'ALGERIA', 'AMERICAN SAMOA', 'ANDORRA', 
                          'ANGOLA', 'ANGUILLA', 'ANTARCTICA', 'ANTIGUA AND BARBUDA', 'ARGENTINA', 
                          'ARMENIA', 'ARUBA', 'AUSTRALIA', 'AUSTRIA', 'AZERBAIJAN', 'BAHAMAS', 
                          'BAHRAIN', 'BANGLADESH', 'BARBADOS', 'BELARUS', 'BELGIUM', 'BELIZE', 
                          'BENIN', 'BERMUDA', 'BHUTAN', 'BOLIVIA', 'BOSNIA AND HERZEGOVINA', 
                          'BOTSWANA', 'BOUVET ISLAND', 'BRAZIL', 'BRITISH INDIAN OCEAN TERRITORY', 
                          'BRUNEI DARUSSALAM', 'BULGARIA', 'BURKINA FASO', 'BURUNDI', 'CAMBODIA', 
                          'CAMEROON', 'CANADA', 'CAPE VERDE', 'CAYMAN ISLANDS', 'CENTRAL AFRICAN REPUBLIC', 
                          'CHAD', 'CHILE', 'CHINA', 'CHRISTMAS ISLAND', 'COCOS (KEELING) ISLANDS', 
                          'COLOMBIA', 'COMOROS', 'CONGO', 'CONGO, THE DEMOCRATIC REPUBLIC OF THE', 
                          'COOK ISLANDS', 'COSTA RICA', "COTE D'IVOIRE", 'CROATIA', 'CUBA', 'CYPRUS', 
                          'CZECH REPUBLIC', 'DENMARK', 'DJIBOUTI', 'DOMINICA', 'DOMINICAN REPUBLIC', 
                          'ECUADOR', 'EGYPT', 'EL SALVADOR', 'EQUATORIAL GUINEA', 'ERITREA', 'ESTONIA', 
                          'ETHIOPIA', 'FALKLAND ISLANDS (MALVINAS)', 'FAROE ISLANDS', 'FIJI', 'FINLAND', 
                          'FRANCE', 'FRENCH GUIANA', 'FRENCH POLYNESIA', 'FRENCH SOUTHERN TERRITORIES', 
                          'GABON', 'GAMBIA', 'GEORGIA', 'GERMANY', 'GHANA', 'GIBRALTAR', 'GREECE', 
                          'GREENLAND', 'GRENADA', 'GUADELOUPE', 'GUAM', 'GUATEMALA', 'GUINEA', 
                          'GUINEA-BISSAU', 'GUYANA', 'HAITI', 'HEARD ISLAND AND MCDONALD ISLANDS', 
                          'HOLY SEE (VATICAN CITY STATE)', 'HONDURAS', 'HONG KONG', 'HUNGARY', 'ICELAND', 
                          'INDIA', 'INDONESIA', 'IRAN, ISLAMIC REPUBLIC OF', 'IRAQ', 'IRELAND', 'ISRAEL', 
                          'ITALY', 'JAMAICA', 'JAPAN', 'JORDAN', 'KAZAKHSTAN', 'KENYA', 'KIRIBATI', 
                          'KOREA, DEMOCRATIC PEOPLE’S REPUBLIC OF', 'KOREA, REPUBLIC OF', 'KUWAIT', 
                          'KYRGYZSTAN', "LAO PEOPLE’S DEMOCRATIC REPUBLIC", 'LATVIA', 'LEBANON', 'LESOTHO', 
                          'LIBERIA', 'LIBYAN ARAB JAMAHIRIYA', 'LIECHTENSTEIN', 'LITHUANIA', 'LUXEMBOURG', 
                          'MACAO', 'MACEDONIA, THE FORMER YUGOSLAV REPUBLIC OF', 'MADAGASCAR', 'MALAWI', 
                          'MALAYSIA', 'MALDIVES', 'MALI', 'MALTA', 'MARSHALL ISLANDS', 'MARTINIQUE', 
                          'MAURITANIA', 'MAURITIUS', 'MAYOTTE', 'MEXICO', 'MICRONESIA, FEDERATED STATES OF', 
                          'MOLDOVA, REPUBLIC OF', 'MONACO', 'MONGOLIA', 'MONTSERRAT', 'MOROCCO', 
                          'MOZAMBIQUE', 'MYANMAR', 'NAMIBIA', 'NAURU', 'NEPAL', 'NETHERLANDS', 
                          'NETHERLANDS ANTILLES', 'NEW CALEDONIA', 'NEW ZEALAND', 'NICARAGUA', 'NIGER', 
                          'NIGERIA', 'NIUE', 'NORFOLK ISLAND', 'NORTHERN MARIANA ISLANDS', 'NORWAY', 
                          'OMAN', 'PAKISTAN', 'PALAU', 'PALESTINIAN TERRITORY, OCCUPIED', 'PANAMA', 
                          'PAPUA NEW GUINEA', 'PARAGUAY', 'PERU', 'PHILIPPINES', 'PITCAIRN', 'POLAND', 
                          'PORTUGAL', 'PUERTO RICO', 'QATAR', 'REUNION', 'ROMANIA', 'RUSSIAN FEDERATION', 
                          'RWANDA', 'SAINT HELENA', 'SAINT KITTS AND NEVIS', 'SAINT LUCIA', 
                          'SAINT PIERRE AND MIQUELON', 'SAINT VINCENT AND THE GRENADINES', 'SAMOA', 
                          'SAN MARINO', 'SAO TOME AND PRINCIPE', 'SAUDI ARABIA', 'SENEGAL', 
                          'SERBIA AND MONTENEGRO', 'SEYCHELLES', 'SIERRA LEONE', 'SINGAPORE', 'SLOVAKIA', 
                          'SLOVENIA', 'SOLOMON ISLANDS', 'SOMALIA', 'SOUTH AFRICA', 
                          'SOUTH GEORGIA AND THE SOUTH SANDWICH ISLANDS', 'SPAIN', 'SRI LANKA', 'SUDAN', 
                          'SURINAME', 'SVALBARD AND JAN MAYEN', 'SWAZILAND', 'SWEDEN', 'SWITZERLAND', 
                          'SYRIAN ARAB REPUBLIC', 'TAIWAN, PROVINCE OF CHINA', 'TAJIKISTAN', 'TANZANIA', 
                          'THAILAND', 'TIMOR-LESTE', 'TOGO', 'TOKELAU', 'TONGA', 'TRINIDAD AND TOBAGO', 
                          'TUNISIA', 'TURKEY', 'TURKMENISTAN', 'TURKS AND CAICOS ISLANDS', 'TUVALU', 'UGANDA', 
                          'UKRAINE', 'UNITED ARAB EMIRATES', 'UNITED KINGDOM', 'UNITED STATES', 
                          'UNITED STATES MINOR OUTLYING ISLANDS', 'URUGUAY', 'UZBEKISTAN', 'VANUATU', 
                          'VENEZUELA', 'VIET NAM', 'VIRGIN ISLANDS, BRITISH', 'VIRGIN ISLANDS, U.S.', 
                          'WALLIS AND FUTUNA', 'WESTERN SAHARA', 'YEMEN', 'ZAMBIA', 'ZIMBABWE',
                          'EIRE','CHANNEL ISLANDS', 'UNSPECIFIED', 'EU', 'RSA'],
}

In [108]:
dim_location = pd.DataFrame(country_data)

dim_location['country_id'] = dim_location['country_id'].astype('Int64')
dim_location['iso_country_code'] = dim_location['iso_country_code'].astype(str)
dim_location['country_name'] = dim_location['country_name'].astype(str)
dim_location['country_code_name'] = dim_location['country_code_name'].astype(str)

dim_location = dim_location[['country_id','iso_country_code', 'country_name', 'country_code_name']]
display(dim_location.head())

,country_id,iso_country_code,country_name,country_code_name
0,1,AF,Afghanistan,AFGHANISTAN
1,2,AL,Albania,ALBANIA
2,3,DZ,Algeria,ALGERIA
3,4,AS,American Samoa,AMERICAN SAMOA
4,5,AD,Andorra,ANDORRA


#### fact_transactions

In [196]:
fact_transactions = transactions.copy()
fact_transactions['transaction_date_id'] = fact_transactions['event_timestamp_invoiced_at'].dt.date.map(
    dim_date.set_index('date')['date_id']
)
fact_transactions['country_id'] = fact_transactions['country_name'].map(
    dim_location.set_index('country_name')['country_id']
)
fact_transactions['date'] = fact_transactions['event_timestamp_invoiced_at'].dt.date
fact_transactions['date_id'] = fact_transactions['date'].map(
    dim_date.set_index('date')['date_id']
)
fact_transactions['total_price_eur'] = fact_transactions['quantity_amount'] * fact_transactions['unit_price_eur']
fact_transactions = fact_transactions[['transaction_id', 'invoice_id', 'event_timestamp_invoiced_at', 'date_id', 'item_uuid', 'item_id', 'transaction_date_id',
                                       'quantity_amount', 'unit_price_eur', 'total_price_eur', 'customer_id', 
                                       'country_id', 'is_return']]

display(fact_transactions.head())

,transaction_id,invoice_id,event_timestamp_invoiced_at,date_id,item_uuid,item_id,transaction_date_id,quantity_amount,unit_price_eur,total_price_eur,customer_id,country_id,is_return
0,0,536365,2010-12-01 08:26:00,700,a5eeac374da2111031f7519f81943391,85123A,700,6,2.55,15.30,17850,225,False
1,1,536365,2010-12-01 08:26:00,700,fe7a790952c55d610228175df6bc6a3c,71053,700,6,3.39,20.34,17850,225,False
2,2,536365,2010-12-01 08:26:00,700,28c1dc0613efda1aeb0113274a9a8dde,84406B,700,8,2.75,22.00,17850,225,False
3,3,536365,2010-12-01 08:26:00,700,70c1999f8fb610f63be227d65201f501,84029G,700,6,3.39,20.34,17850,225,False
4,4,536365,2010-12-01 08:26:00,700,cec12c5a55ddfa8fb7eb3ae3b24c8c46,84029E,700,6,3.39,20.34,17850,225,False


## ETL logic

In [179]:
def createIntItems(fact_transactions, dim_items, dim_item_family):
    int_items = fact_transactions \
        .merge(dim_items[['item_uuid', 'item_description', 'item_family_id', 
                          'is_operational_item', 'is_unknown_item', 
                          'is_special_item', 'is_modification_item', 'is_error_item']], on='item_uuid') \
        .merge(dim_item_family[['item_family_id', 'item_family_description', 'item_category']], on='item_family_id') \
        .groupby(['item_uuid', 'item_description', 'item_family_id', 'item_family_description', 
                  'item_category', 'is_operational_item', 'is_unknown_item', 
                  'is_special_item', 'is_modification_item', 'is_error_item'], as_index=False) \
        .agg(total_sales_amount=('total_price_eur', 'sum'),
             total_customers=('customer_id', 'nunique'),
             total_quantity=('quantity_amount', 'sum'),
             total_transactions=('transaction_id', 'nunique'))
    return int_items

In [180]:
int_items = createIntItems(fact_transactions, dim_items, dim_item_family)
display(int_items)

,item_uuid,item_description,item_family_id,item_family_description,item_category,is_operational_item,is_unknown_item,is_special_item,is_modification_item,is_error_item,total_sales_amount,total_customers,total_quantity,total_transactions
0,00101644c59f5ca2a40eba1ec330267f,IVORY ENCHANTED FOREST PLACEMAT,85114,ENCHANTED FOREST PLACEMAT,TABLEWARE,False,False,False,False,False,900.84,36,476,79
1,0011274a066e1cd254cbb1f466b8fc70,LARGE PARLOUR FRAME,23457,LARGE PARLOUR FRAME,HOME DECOR,False,False,False,False,False,122.85,2,27,2
2,00337bc60cace50c98618a90a70682ca,RED RETROSPOT APRON,21042,RED RETROSPOT APRON,KITCHENWARE,False,False,False,False,False,2171.07,72,329,140
3,0041175d5926046b2323f63772a8c706,nan,84789,ENCHANTED BIRD PLANT CAGE,GARDENING,False,True,False,False,False,0.00,0,-1,2
4,006e6672caa869d2f012cca839a92dc0,nan,21054,NURSE'S BAG SOFT TOY,TOYS,False,True,False,False,False,0.00,0,-29,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5611,ffbfba04d5c6959e3ea33f0c5526dce6,FANCY FONTS BIRTHDAY WRAP,21497,FANCY FONTS BIRTHDAY WRAP,STATIONERY,False,False,False,False,False,3430.00,177,8600,232
5612,ffcdd749173f9f1287bbde555a310021,nan,82567,"AIRLINE LOUNGE,METAL SIGN",HOME DECOR,False,True,False,False,False,0.00,0,-73,1
5613,ffd32fc8cc7eda3c29aec9b3b56c38ff,HAND PAINTED HANGING EASTER EGG,23441,HAND PAINTED HANGING EASTER EGG,HOLIDAY,False,False,False,False,False,9.24,3,22,3
5614,fff4b160bd4b6c49df6bcd171bc53f26,BICYCLE SAFTEY WALL ART,23535,BICYCLE WALL ART,HOME DECOR,False,False,False,False,False,2371.94,38,458,39


In [181]:
def createIntCustomer(fact_transactions, dim_customers, dim_location, int_items):
    fact_transactions = fact_transactions.merge(
        int_items[['item_uuid', 'item_category']], on='item_uuid', how='left'
    )

    customer_summary = fact_transactions \
        .merge(dim_customers[['customer_id']], on='customer_id') \
        .merge(dim_location[['country_id', 'country_name']], on='country_id') \
        .groupby(['customer_id', 'country_name'], as_index=False) \
        .agg(
            total_sales_amount=('total_price_eur', 'sum'),
            total_quantity=('quantity_amount', 'sum'),
            total_transactions=('transaction_id', 'nunique'),
            total_invoices=('invoice_id', 'nunique'),
            total_returns=('is_return', 'sum'),
            favorite_category=('item_category', lambda x: x.mode()[0] if not x.mode().empty else None)
        )
    customer_summary['customer_uuid'] = customer_summary.apply(lambda row: generate_hash(row, ['customer_id', 'country_name']), axis=1)
    customer_summary['avg_sales_per_transaction'] = (customer_summary['total_sales_amount'] / customer_summary['total_transactions']).round(2)
    customer_summary = customer_summary[[
        'customer_uuid', 'customer_id', 'country_name', 'total_sales_amount', 
        'total_quantity', 'total_transactions', 'total_invoices', 'total_returns', 
        'favorite_category', 'avg_sales_per_transaction'
    ]]
    return customer_summary

In [182]:
int_customers = createIntCustomer(fact_transactions, dim_customers, dim_location, int_items)
display(int_customers.head())

,customer_uuid,customer_id,country_name,total_sales_amount,total_quantity,total_transactions,total_invoices,total_returns,favorite_category,avg_sales_per_transaction
0,39bc00f0eee6299dc1e7f6e08924c3c0,12346,United Kingdom,0.00,0,2,2,1,STORAGE,0.00
1,6f8b23dd0a5f96a9fc9fb5795817ec77,12347,Iceland,4310.00,2458,182,7,0,HOME DECOR,23.68
2,9f5260dd8517028ddb442d3cc0bda4ae,12348,Finland,1797.24,2341,31,4,0,BAKING SUPPLIES,57.98
3,45cac541ed822ceb5d3924a3458e4308,12349,Italy,1757.55,631,73,1,0,HOME DECOR,24.08
4,e8c1cb158db3be9e61a740dcad06e3d4,12350,Norway,334.40,197,17,1,0,HOME DECOR,19.67


In [209]:
def createIntTransactions(fact_transactions, dim_items, dim_item_family, dim_date, dim_location):
    return fact_transactions \
        .merge(dim_items[['item_uuid', 'item_description', 'item_family_id', 'item_variant','is_operational_item','is_unknown_item','is_special_item','is_modification_item','is_error_item']], on='item_uuid', how = 'left') \
        .merge(dim_item_family[['item_family_id', 'item_family_description', 'item_category']], on='item_family_id', how = 'left') \
        .merge(dim_location[['country_id', 'country_name']], on='country_id', how = 'left') \
        .merge(dim_date[['date_id', 'date', 'day_of_week']], 
               left_on='date_id', right_on='date_id', how = 'left') \
        [['transaction_id', 'invoice_id', 'item_uuid', 'item_id', 'item_description', 'item_family_description', 'item_category', 
          'item_variant', 'is_operational_item','is_unknown_item','is_special_item','is_modification_item','is_error_item', 'unit_price_eur', 'total_price_eur', 'quantity_amount', 'is_return', 'customer_id', 'country_id',
          'country_name', 'date_id', 'date', 'day_of_week']]


In [211]:
int_transactions = createIntTransactions(fact_transactions, dim_items, dim_item_family, dim_date, dim_location)
display(int_transactions.head())

,transaction_id,invoice_id,item_uuid,item_id,item_description,item_family_description,item_category,item_variant,is_operational_item,is_unknown_item,is_special_item,is_modification_item,is_error_item,unit_price_eur,total_price_eur,quantity_amount,is_return,customer_id,country_id,country_name,date_id,date,day_of_week
0,0,536365,a5eeac374da2111031f7519f81943391,85123A,WHITE HANGING HEART T-LIGHT HOLDER,HANGING HEART T-LIGHT HOLDER,CANDLES,A,False,False,False,False,False,2.55,15.30,6,False,17850,225,United Kingdom,700,2010-12-01,Wednesday
1,1,536365,fe7a790952c55d610228175df6bc6a3c,71053,WHITE METAL LANTERN,WHITE METAL LANTERN,LIGHTING,NaN,False,False,False,False,False,3.39,20.34,6,False,17850,225,United Kingdom,700,2010-12-01,Wednesday
2,2,536365,28c1dc0613efda1aeb0113274a9a8dde,84406B,CREAM CUPID HEARTS COAT HANGER,CREAM CUPID HEARTS COAT HANGER,HOME DECOR,B,False,False,False,False,False,2.75,22.00,8,False,17850,225,United Kingdom,700,2010-12-01,Wednesday
3,3,536365,70c1999f8fb610f63be227d65201f501,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,RED WOOLLY HOTTIE WHITE HEART. KNITTED UNION F...,TEXTILES,G,False,False,False,False,False,3.39,20.34,6,False,17850,225,United Kingdom,700,2010-12-01,Wednesday
4,4,536365,cec12c5a55ddfa8fb7eb3ae3b24c8c46,84029E,RED WOOLLY HOTTIE WHITE HEART.,RED WOOLLY HOTTIE WHITE HEART. KNITTED UNION F...,TEXTILES,E,False,False,False,False,False,3.39,20.34,6,False,17850,225,United Kingdom,700,2010-12-01,Wednesday


In [219]:
int_customers

,customer_uuid,customer_id,country_name,total_sales_amount,total_quantity,total_transactions,total_invoices,total_returns,favorite_category,avg_sales_per_transaction
0,39bc00f0eee6299dc1e7f6e08924c3c0,12346,United Kingdom,0.00,0,2,2,1,STORAGE,0.00
1,6f8b23dd0a5f96a9fc9fb5795817ec77,12347,Iceland,4310.00,2458,182,7,0,HOME DECOR,23.68
2,9f5260dd8517028ddb442d3cc0bda4ae,12348,Finland,1797.24,2341,31,4,0,BAKING SUPPLIES,57.98
3,45cac541ed822ceb5d3924a3458e4308,12349,Italy,1757.55,631,73,1,0,HOME DECOR,24.08
4,e8c1cb158db3be9e61a740dcad06e3d4,12350,Norway,334.40,197,17,1,0,HOME DECOR,19.67
...,...,...,...,...,...,...,...,...,...,...
4375,ecf2373a824e198de70bbea44fbaca31,18280,United Kingdom,180.60,45,10,1,0,HOME DECOR,18.06
4376,93dc52ad388fd6e8eafc4dd0ce6420d7,18281,United Kingdom,80.82,54,7,1,0,GIFT ITEMS,11.55
4377,48ff6cef14b9afae119c5fabbd2c2698,18282,United Kingdom,176.60,98,13,3,1,TABLEWARE,13.58
4378,b616da0773cecdadb0ab14cc79b87e09,18283,United Kingdom,2094.88,1397,756,16,0,BAGS,2.77


In [230]:
int_transactions

,transaction_id,invoice_id,item_uuid,item_id,item_description,item_family_description,item_category,item_variant,is_operational_item,is_unknown_item,is_special_item,is_modification_item,is_error_item,unit_price_eur,total_price_eur,quantity_amount,is_return,customer_id,country_id,country_name,date_id,date,day_of_week
0,0,536365,a5eeac374da2111031f7519f81943391,85123A,WHITE HANGING HEART T-LIGHT HOLDER,HANGING HEART T-LIGHT HOLDER,CANDLES,A,False,False,False,False,False,2.55,15.30,6,False,17850,225,United Kingdom,700,2010-12-01,Wednesday
1,1,536365,fe7a790952c55d610228175df6bc6a3c,71053,WHITE METAL LANTERN,WHITE METAL LANTERN,LIGHTING,NaN,False,False,False,False,False,3.39,20.34,6,False,17850,225,United Kingdom,700,2010-12-01,Wednesday
2,2,536365,28c1dc0613efda1aeb0113274a9a8dde,84406B,CREAM CUPID HEARTS COAT HANGER,CREAM CUPID HEARTS COAT HANGER,HOME DECOR,B,False,False,False,False,False,2.75,22.00,8,False,17850,225,United Kingdom,700,2010-12-01,Wednesday
3,3,536365,70c1999f8fb610f63be227d65201f501,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,RED WOOLLY HOTTIE WHITE HEART. KNITTED UNION F...,TEXTILES,G,False,False,False,False,False,3.39,20.34,6,False,17850,225,United Kingdom,700,2010-12-01,Wednesday
4,4,536365,cec12c5a55ddfa8fb7eb3ae3b24c8c46,84029E,RED WOOLLY HOTTIE WHITE HEART.,RED WOOLLY HOTTIE WHITE HEART. KNITTED UNION F...,TEXTILES,E,False,False,False,False,False,3.39,20.34,6,False,17850,225,United Kingdom,700,2010-12-01,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,541904,581587,5c4e9ed3e139204e8b11e2cd8386c304,22613,PACK OF 20 SPACEBOY NAPKINS,PACK OF 20 SPACEBOY NAPKINS,TABLEWARE,NaN,False,False,False,False,False,0.85,10.20,12,False,12680,73,France,1073,2011-12-09,Friday
541905,541905,581587,e20e8d442cae0313dee41fef04d83b95,22899,CHILDREN'S APRON DOLLY GIRL,CHILDREN'S APRON DOLLY GIRL,CHILDREN,NaN,False,False,False,False,False,2.10,12.60,6,False,12680,73,France,1073,2011-12-09,Friday
541906,541906,581587,9edac528da06859f636ce7d9bb5e531b,23254,CHILDRENS CUTLERY DOLLY GIRL,CUTLERY DOLLY GIRL,TABLEWARE,NaN,False,False,False,False,False,4.15,16.60,4,False,12680,73,France,1073,2011-12-09,Friday
541907,541907,581587,4bb4e824ec9cc976142570345f4a1d57,23255,CHILDRENS CUTLERY CIRCUS PARADE,CHILDRENS CUTLERY CIRCUS PARADE,TABLEWARE,NaN,False,False,False,False,False,4.15,16.60,4,False,12680,73,France,1073,2011-12-09,Friday


In [233]:
import pandas as pd

def createIntInvoices(int_transactions, dim_customers, dim_items, dim_location, dim_date):
    invoices = int_transactions.copy()
    int_invoices = invoices.groupby(['invoice_id', 'date', 'customer_id', 'country_name'], as_index=False).agg(
        date_id=('date_id', 'first'),
        date=('date', 'first'),
        day_of_week=('day_of_week', 'first'),
        country_id=('country_id', 'first'),
        total_invoice_amount=('total_price_eur', 'sum'),
        total_quantity=('quantity_amount', 'sum'),
        total_items=('item_uuid', 'nunique'),
        has_return=('is_return', 'any'),
        has_operational_item=('is_operational_item', 'any'),
        has_unknown_item=('is_unknown_item', 'any'),
        has_special_item=('is_special_item', 'any'),
        has_modification_item=('is_modification_item', 'any'),
        has_error_item=('is_error_item', 'any')
    )
    
    return int_invoices


In [234]:
int_invoices = createIntInvoices(int_transactions, dim_customers, dim_items, dim_location, dim_date)
display(int_invoices.head())

,invoice_id,customer_id,country_name,date_id,date,day_of_week,country_id,total_invoice_amount,total_quantity,total_items,has_return,has_operational_item,has_unknown_item,has_special_item,has_modification_item,has_error_item
0,536365,17850,United Kingdom,700,2010-12-01,Wednesday,225,139.12,40,7,False,False,False,False,False,False
1,536366,17850,United Kingdom,700,2010-12-01,Wednesday,225,22.20,12,2,False,False,False,False,False,False
2,536367,13047,United Kingdom,700,2010-12-01,Wednesday,225,278.73,83,12,False,False,False,False,False,False
3,536368,13047,United Kingdom,700,2010-12-01,Wednesday,225,70.05,15,4,False,False,False,False,False,False
4,536369,13047,United Kingdom,700,2010-12-01,Wednesday,225,17.85,3,1,False,False,False,False,False,False
